Importing librarys

In [3]:
import numpy as np
import gym

Setting Up environment

In [4]:
env = gym.make('MountainCar-v0')

Env variables

In [ ]:
space =  env.observation_space
actions = env.action_space